# 01 - Ollama Tool Loop Skeleton

This notebook is a minimal Python orchestration loop for an LLM DM using Ollama tools + JSON schema.

It mirrors the key pattern from this repo:
- build prompt context
- call model
- execute tool calls
- append tool results
- repeat until no tool calls


In [ ]:
import json
import random
import re
from typing import Any

import requests

OLLAMA_URL = "http://localhost:11434/api/chat"
MODEL = "llama3.1:8b-instruct"  # change to your local model


In [ ]:
world_state = {
    "location": "Ravenhill",
    "time_of_day": "dusk",
    "weather": "foggy",
    "active_quests": ["Find the missing miller"],
}

def get_scene_state() -> dict[str, Any]:
    return world_state

def set_scene_state(location: str | None = None, time_of_day: str | None = None, weather: str | None = None) -> dict[str, Any]:
    if location:
        world_state["location"] = location
    if time_of_day:
        world_state["time_of_day"] = time_of_day
    if weather:
        world_state["weather"] = weather
    return world_state

def roll_dice(expression: str = "1d20") -> dict[str, Any]:
    m = re.fullmatch(r"(\d+)d(\d+)([+-]\d+)?", expression.strip())
    if not m:
        raise ValueError(f"Invalid dice expression: {expression}")
    n, sides, mod = int(m.group(1)), int(m.group(2)), int(m.group(3) or 0)
    rolls = [random.randint(1, sides) for _ in range(n)]
    return {"expression": expression, "rolls": rolls, "modifier": mod, "total": sum(rolls) + mod}

def add_quest(title: str) -> dict[str, Any]:
    world_state.setdefault("active_quests", []).append(title)
    return world_state

TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "get_scene_state",
            "description": "Read the current DM world state.",
            "parameters": {"type": "object", "properties": {}, "required": []},
        },
    },
    {
        "type": "function",
        "function": {
            "name": "set_scene_state",
            "description": "Update location/time/weather.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string"},
                    "time_of_day": {"type": "string"},
                    "weather": {"type": "string"},
                },
                "required": [],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "roll_dice",
            "description": "Roll dice in NdM(+/-K) format, e.g. 1d20+3.",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {"type": "string", "description": "Dice expression"}
                },
                "required": ["expression"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "add_quest",
            "description": "Add a new quest objective to the active quest log.",
            "parameters": {
                "type": "object",
                "properties": {
                    "title": {"type": "string"}
                },
                "required": ["title"],
            },
        },
    },
]

TOOL_IMPL = {
    "get_scene_state": get_scene_state,
    "set_scene_state": set_scene_state,
    "roll_dice": roll_dice,
    "add_quest": add_quest,
}


In [ ]:
def ollama_chat(messages: list[dict[str, Any]], tools: list[dict[str, Any]] | None = None, model: str = MODEL) -> tuple[dict[str, Any], dict[str, Any]]:
    payload: dict[str, Any] = {
        "model": model,
        "messages": messages,
        "stream": False,
    }
    if tools:
        payload["tools"] = tools

    r = requests.post(OLLAMA_URL, json=payload, timeout=120)
    r.raise_for_status()
    data = r.json()
    return data["message"], data


def extract_tool_calls(message: dict[str, Any]) -> list[dict[str, Any]]:
    out: list[dict[str, Any]] = []
    for i, call in enumerate(message.get("tool_calls") or []):
        fn = call.get("function", {})
        name = fn.get("name") or call.get("name")
        args = fn.get("arguments", {})
        if isinstance(args, str):
            try:
                args = json.loads(args)
            except json.JSONDecodeError:
                args = {}

        out.append({
            "id": call.get("id") or f"call_{i}",
            "name": name,
            "arguments": args,
        })
    return out


def execute_tool(name: str, arguments: dict[str, Any]) -> dict[str, Any]:
    if name not in TOOL_IMPL:
        return {"ok": False, "error": f"Unknown tool: {name}"}
    try:
        result = TOOL_IMPL[name](**arguments)
        return {"ok": True, "result": result}
    except Exception as e:
        return {"ok": False, "error": str(e)}


In [ ]:
SYSTEM_PROMPT = (
    "You are a DnD dungeon master. Keep narrative vivid but grounded in world state. "
    "Use tools whenever world state lookup/update or dice is needed. "
    "When done, provide a concise scene update and 2-3 player choices."
)


def run_tool_loop(user_prompt: str, max_iterations: int = 10) -> dict[str, Any]:
    messages: list[dict[str, Any]] = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_prompt},
    ]
    rounds: list[dict[str, Any]] = []

    for i in range(max_iterations):
        assistant, raw = ollama_chat(messages, TOOLS)
        assistant_text = assistant.get("content", "")
        tool_calls = extract_tool_calls(assistant)

        rounds.append({
            "iteration": i + 1,
            "assistant_text": assistant_text,
            "tool_calls": tool_calls,
            "done_reason": raw.get("done_reason"),
        })

        if not tool_calls:
            return {
                "status": "completed",
                "final_answer": assistant_text,
                "rounds": rounds,
                "messages": messages,
            }

        messages.append({
            "role": "assistant",
            "content": assistant_text,
            "tool_calls": assistant.get("tool_calls", []),
        })

        for call in tool_calls:
            tool_result = execute_tool(call["name"], call["arguments"])
            messages.append({
                "role": "tool",
                "tool_name": call["name"],
                "content": json.dumps(tool_result),
            })

    return {
        "status": "max_iterations",
        "final_answer": "Stopped: hit max iterations before model produced a final narrative response.",
        "rounds": rounds,
        "messages": messages,
    }


In [ ]:
# Make sure Ollama is running and the model is pulled first.
# Example: ollama pull llama3.1:8b-instruct

result = run_tool_loop("The party arrives at Ravenhill and asks if they can track the missing miller through the fog.")

print("status:", result["status"])
print("rounds:", len(result["rounds"]))
print("\nFINAL ANSWER\n")
print(result["final_answer"])

print("\nTOOL CALL TRACE\n")
for r in result["rounds"]:
    names = [c["name"] for c in r["tool_calls"]]
    print(f"iter={r['iteration']} tools={names}")
